In [1]:
import sys
sys.path.insert(0, '../genepattern-python/')
from gp import gp_functions

%load_ext autoreload
%autoreload 2

In [2]:
gp_functions.login('https://genepattern.broadinstitute.org/gp', 'Simpli', 'Simpli')

gp_functions.get_tasks()

In [16]:
d = {
    "library_path": "path/to/library/",  # Optional

    "tasks":  # Required
        [
            {
                "label": "Name of this Task (unique ID)",  # Optional (will be created based on function name if not specified)
                "description": "This Task performs ...",  # Optional
                "function_path": "library.path.to.file.function",  # Optional (function is assumed to be in the __main__ if not specified)
                "required_args":  # Optional
                    [
                        {
                            "arg_name": "x",  # Required if specifying required_args
                            "label": "The X",  # Optional if specifying required_args (user sees label instead of arg_name in a Task Widget) (will be created based on arg_name if not specified)
                            "description": "The X is ...",  # Optional if specifying required_args
                        },
                    ],
                "optional_args":  # Optional
                    [
                        {
                            "arg_name": "z",  # Required if specifying optional_args
                            "label": "The Z",  # Optional if specifying optional_args (user sees label instead of arg_name in a Task Widget) (will be created based on arg_name if not specified)
                            "description": "The Z is ...",  # Optional if specifying optional_args
                        },
                    ],
                "returns":  # Optional
                    [
                        {
                            "label": "The A",  # Required
                            "description": "The A is ...",  # Optional if specifying returns
                        },
                    ],
                "other_information":  # Optional
                    {
                        "version": ["1.0.0"],
                        "server": ["Broad Institute"],
                        "tag": ["RNA", "Sequencing"],
                    },
            },
        ]
}

In [4]:
t = gp_functions.GPTask(gp_functions.SERVER, 'PreprocessDataset')

t.param_load()
js = t.make_job_spec()

print('Task: {}'.format(t.get_name()))
print('\tLSID: {}'.format(t.get_lsid()))
print('\tDescription: {}'.format(t.get_description()))

ps = t.get_parameters()
for p in ps:
    print('\tParameter: {}'.format(p.get_name()))
    print('\t\tDescription: {}'.format(p.get_description()))
    print('\t\tType: {}'.format(p.get_type()))
    print('\t\tDefault Value: {}'.format(p.get_default_value()))
    print('\t\tOptional: {}'.format(p.is_optional()))
    if p.is_choice_param():
        
        print('\t\tChoices: {}'.format([d['value'] for d in p.get_choices()]))
        
    if p.get_default_value():
        js.set_parameter(p.get_name(), p.get_default_value())
        print('\t\t{}={}\n'.format(p.get_name(), p.get_default_value()))
    else:
        print('\t\t*********\n\t\t{}=\n\t\t*********\n'.format(p.get_name()))

Task: PreprocessDataset
	LSID: urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00020:5.1
	Description: Performs several preprocessing steps on a res, gct, or odf input file
	Parameter: input.filename
		Description: Input filename - .res, .gct, .odf
		Type: File
		Default Value: None
		Optional: False
		*********
		input.filename=
		*********

	Parameter: threshold.and.filter
		Description: Apply floor, ceiling and variation filter
		Type: String
		Default Value: 1
		Optional: False
		Choices: ['0', '1']
		threshold.and.filter=1

	Parameter: floor
		Description: Value for floor threshold
		Type: String
		Default Value: 20
		Optional: False
		floor=20

	Parameter: ceiling
		Description: Value for ceiling threshold
		Type: String
		Default Value: 20000
		Optional: False
		ceiling=20000

	Parameter: min.fold.change
		Description: Minimum fold change for variation filter
		Type: String
		Default Value: 3
		Optional: False
		min.fold.change=3

	Parameter: min.delta
		Descr

In [5]:
js.set_parameter('input.filename',
                 'https://software.broadinstitute.org/cancer/software/genepattern/data/all_aml/all_aml_test.gct')

In [6]:
j = gp_functions.submit_job(js)

b'{"lsid": "urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00020:5.1", "params": [{"name": "threshold.and.filter", "values": ["1"]}, {"name": "floor", "values": ["20"]}, {"name": "ceiling", "values": ["20000"]}, {"name": "min.fold.change", "values": ["3"]}, {"name": "min.delta", "values": ["100"]}, {"name": "num.outliers.to.exclude", "values": ["0"]}, {"name": "row.normalization", "values": ["0"]}, {"name": "row.sampling.rate", "values": ["1"]}, {"name": "log2.transform", "values": ["0"]}, {"name": "output.file.format", "values": ["3"]}, {"name": "output.file", "values": ["<input.filename_basename>.preprocessed"]}, {"name": "input.filename", "values": ["https://software.broadinstitute.org/cancer/software/genepattern/data/all_aml/all_aml_test.gct"]}], "tags": ["GenePattern Python Client"]}'


In [9]:
j.get_status_message()

'The job is queued or being scheduled and executed'

In [10]:
j.is_finished()

True

In [14]:
for f in j.get_output_files():
    print(f.get_url())
    data = f.read()

https://genepattern.broadinstitute.org/gp/jobResults/1452771/all_aml_test.preprocessed.gct


In [15]:
data

'#1.2\n5951\t35\nName\tDescription\tALL 19769 TA+ Norel\tALL 406 TA+ (ML) Norel\tALL 4466 Norel\tALL 1245 TA- Norel\tALL 16125 TA- Norel\tALL 23368 TA- Norel\tALL R28 (ML) Relap\tALL 1234 ML\tALL 1305 ML\tALL 1421 ML\tALL 1425 ML\tALL 1486 ML\tALL 1457 ML\tALL 1256 TA+ (ML) Norel\tALL 1275 TA+ (ML) Norel\tALL 1101 TA- (ML) Norel\tALL 1319 TA- (ML) Norel\tALL 522 TA- (ML) Norel\tALL SH 6\tALL SH 12\tALL SH 15\tAML 15 (PK) Norel\tAML 19 (PK) Norel\tAML 10 (PK) Relap\tAML 9 (PK) Relap\tAML SH 5\tAML SH 13\tAML SH 14\tAML SH 16\tAML SH 18\tAML 17 ML\tAML 25 ML\tAML 11 (ML)\tAML 12 (ML)\tAML 5 (ML) Relap\nAFFX-BioB-3_at\tAFFX-BioB-3_at (endogenous control)\t20.0\t41.0\t262.0\t20.0\t20.0\t20.0\t20.0\t20.0\t20.0\t170.0\t312.0\t43.0\t20.0\t23.0\t20.0\t49.0\t20.0\t20.0\t20.0\t59.0\t20.0\t20.0\t59.0\t24.0\t212.0\t102.0\t20.0\t20.0\t38.0\t20.0\t20.0\t63.0\t20.0\t20.0\t20.0\nAFFX-BioC-5_at\tAFFX-BioC-5_at (endogenous control)\t88.0\t328.0\t295.0\t276.0\t182.0\t266.0\t24.0\t142.0\t252.0\t174.0\t142